In [101]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import json
import re
from uO_scrape import get_offerings, is_activity, get_sections
from time import sleep, perf_counter as pf
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += ':DES-CBC3-SHA'

def raze_list(l):
    out = []
    for x in l:
        if type(x) == list:
            out += raze_list(x)
        else:
            out.append(x)
    return out

In [1]:
def get_files():
    x = !dir
    files = [y.rsplit(' ', 1)[-1] for y in list(x)]
    files = [x for x in files if '.' in x]
    return files

files = get_files()
files

['.',
 '..',
 '.ipynb_checkpoints',
 'get_schedules.ipynb',
 'get_schedules.py',
 'get_subjects.ipynb',
 'read_subjects.ipynb',
 'test_get_offers.ipynb',
 'uOttawa_courses.csv',
 'uOttawa_courses.json',
 'uOttawa_subjects.csv']

# Quick Get-All

In [4]:
courses = get_offerings('', '')

In [8]:
table = pd.read_json(json.dumps(courses), orient='records')
table.code = table.code.str.strip().str[:3] + ' ' + table.code.str.strip().str[-4:]
print('\ttable', '-', 'unique codes')
print('\t', len(table), '-', len(set(table.code)))
table.head()

	table - unique codes
	 2882 - 2615


,code,department,faculty,link,term,title
0,ADM 1100,Administration (General),Telfer School of Management,Course.aspx?id=000001&term=2189&session=FS,2018 Fall Term,Introduction to Business Management
1,ADM 1101,Administration (General),Telfer School of Management,Course.aspx?id=000002&term=2189&session=FS,2018 Fall Term,Social Context of Business
2,ADM 1300,Administration (General),Telfer School of Management,Course.aspx?id=000003&term=2189&session=FS,2018 Fall Term,Introduction to Business Management
3,ADM 1301,Administration (General),Telfer School of Management,Course.aspx?id=000004&term=2189&session=FS,2018 Fall Term,Social Context of Business
4,ADM 1340,Administration (General),Telfer School of Management,Course.aspx?id=000005&term=2189&session=FS,2018 Fall Term,Financial Accounting


In [9]:
table.to_csv('uOttawa_offerings_quick.csv', index=False, mode='a', header=not('uOttawa_offerings_quick.csv' in files))

# Long Get-All

In [20]:
def get_course_offerings(courses, save=None):
    '''
    Gets offerings for each course in courses individually, saving to save so as not to lose progress
    '''
    s = requests.Session()
    output = []
    temp = []
    try:
        for course in courses:
            t1 = pf()
            temp = get_offerings(course, session=s)
            if save is not None:
                pd.read_json(json.dumps(temp), orient='records').to_csv(save, \
                        index=False, header=not(save in get_files()), mode='a')
            output += temp
            t2 = pf()
            sleep(max(1-(t2-t1), 0))
    except KeyboardInterrupt:
        output += temp
        if save is not None:
            pd.read_json(json.dumps(temp), orient='records').to_csv(save, \
                index=False, header=not(save in get_files()), mode='a')
        return output
    return output

In [18]:
course_table = pd.read_csv('uOttawa_courses.csv', index_col=False)
courses = course_table.code.tolist()
course_table.head()

,code,components,credits,dependencies,desc,prerequisites,title
0,EAS 1101,['Lecture'],3,[],"Introduction to the global, cultural and relig...",NaN,Introduction to Aboriginal Societies and Cultures
1,EAS 1501,['Cours magistral'],3,[],Introduction à la diversité culturelle et reli...,NaN,Introduction aux sociétés et cultures autochtones
2,EAS 2101,['Lecture'],3,[],Study of the concept and history of colonialis...,NaN,Colonialism and Indigenous Peoples
3,EAS 2501,['Cours magistral'],3,[],Étude conceptuelle et historique du colonialis...,NaN,Colonialisme et peuples autochtones
4,EAS 3101,['Lecture'],3,['EAS 2101'],Study and analysis of methodologies associated...,EAS 2101 or 54 university credits,Research and Methodologies in Aboriginal Studies


In [22]:
#finds courses whose offerings have not already been retrieved
if 'uOttawa_offerings_long.csv' in files:
    done_courses = set(pd.read_csv('uOttawa_offerings_long.csv', index_col=False).code.tolist())
    courses = list(set(courses).difference(done_courses))
print('\tGetting offerings')
#picks up where it was left off and continues searching individually for course offerings
offerings = get_course_offerings(courses, 'uOttawa_offerings_long.csv')

	Getting offerings


In [23]:
offerings_table = pd.read_json(json.dumps(offerings), orient='records')
offerings_table.head()

,code,department,faculty,link,term,title
0,EAS1101,Arts (General),Faculty of Arts,Course.aspx?id=009180&term=2189&session=FS,2018 Fall Term,Introduction to Aboriginal Societies and Cultures
1,MAT1300,Department of Mathematics and Statistics,Faculty of Science,Course.aspx?id=019732&term=2189&session=FS,2018 Fall Term,Mathematical Methods I
2,MAT1300,Department of Mathematics and Statistics,Faculty of Science,Course.aspx?id=019732&term=2191&session=FS,2019 Winter Term,Mathematical Methods I
3,MAT1700,Department of Mathematics and Statistics,Faculty of Science,Course.aspx?id=019762&term=2189&session=FS,2018 Fall Term,Méthodes mathématiques I
4,MAT1700,Department of Mathematics and Statistics,Faculty of Science,Course.aspx?id=019762&term=2191&session=FS,2019 Winter Term,Méthodes mathématiques I


# Get Specific Courses

# Getting Schedules

In [127]:
link = table[table.code.str.contains('MAT 2141')].link.tolist()[0]
sections = get_sections(link)
sections

[[{'section': 'MAT2141 A00(September 05 - December 05)',
   'activity': 'LEC',
   'time': 'Thursday 13:00 - 14:20',
   'location': 'VNR 3035',
   'professor': 'Alistair  Savage'},
  {'section': 'MAT2141 A00(September 05 - December 05)',
   'activity': 'LEC',
   'time': 'Monday 11:30 - 12:50',
   'location': 'VNR 3035',
   'professor': 'Alistair  Savage'},
  {'section': 'MAT2141 A01(September 05 - December 05)',
   'activity': 'DGD',
   'time': 'Wednesday 10:00 - 11:20',
   'location': 'CBY B012',
   'professor': 'Not available at this time.'}]]